# Generating Adversarial Dataset

Generating Adversarial Dataset using Model

## Imports

### GPU Information

The following GPU is used while Training the Model. Google Colab is used for Training so the GPU might change while running it later.

**GPU-Specifications:**
*   Name: Tesla P100-PCIE
*   GPU Memory: 16280MiB

In [1]:
!nvidia-smi

Tue Aug 24 17:09:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Importing Data
Files are uploaded in Google Drive. Notebook is connected to Google Drive

In [2]:
from google.colab import drive 
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/Modulation-Classification/RadioML-Dataset/Data/CW-Attack

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/Modulation-Classification/RadioML-Dataset/Data/CW-Attack


### Importing Libraries

Cleverhans is used for generating Adversarial Examples.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as pimg
import seaborn as sns
import scipy.io
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score

# Tensorflow Libraries
!pip3 install tensorflow==2.2.0
!pip3 install keras-tuner==1.0.2
import tensorflow as tf
#tf.compat.v1.disable_eager_execution()
from tensorflow.keras.layers import Dense, Flatten, ReLU, Conv2D, Input, MaxPooling2D, BatchNormalization, AveragePooling2D, Reshape, ZeroPadding2D, Dropout, Activation, Permute, Concatenate
from tensorflow.keras.models import Model,Sequential

# ART Libraries
"""
!pip3 install adversarial-robustness-toolbox[tensorflow]
from art.attacks.evasion import CarliniL2Method
from art.estimators.classification import KerasClassifier
"""

# Cleverhans Libraries
!pip3 install cleverhans
from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method
from carlini_wagner_l2 import CarliniWagnerL2

### Importing Data

In [4]:
# Loading Training Data
Clean_X_Train = np.load("../Clean-Data/Clean_X_Train.npy")
Clean_y_Train = np.load("../Clean-Data/Clean_y_Train.npy")

# Loading Validation Data
f = open("../Clean-Data/Clean_X_Valid.pkl","rb")
Clean_X_Valid = pickle.load(f)

f = open("../Clean-Data/Clean_y_Valid.pkl","rb")
Clean_y_Valid = pickle.load(f)

## Designing Attack

**CW Attack**
```
CarliniWagnerL2(
            model_fn = Robust_CNN,
            y = y_Batch,
            targeted = False,
            batch_size = Batch_Size,
            clip_max = Max,
            clip_min = Min,
            max_iterations = nb_Iterations,
            confidence = Confidence,
            learning_rate = Learning_Rate
        )

X_Batch_Adv = CW.attack(X_Batch)
```

Modulation Schemes

In [5]:
f = open("../Dataset/ModulationMap.pkl", "rb")
ModulationMap = pickle.load(f)
ModulationMap = dict((v,k) for k,v in ModulationMap.items())
ModulationMap

{'8PSK': 0,
 'AM-DSB': 1,
 'AM-SSB': 2,
 'BPSK': 3,
 'CPFSK': 4,
 'GFSK': 5,
 'PAM4': 6,
 'QAM16': 7,
 'QAM64': 8,
 'QPSK': 9,
 'WBFM': 10}

Attack Settings




In [6]:
# Parameters of CW
nb_Iterations = 500
Confidence = 0.9
Batch_Size = 200
Learning_Rate = 0.01
Max = np.max(Clean_X_Train)
Min = np.min(Clean_X_Train)

## Generation of Adversarial Dataset

Loading Model

In [7]:
# Model
Robust_CNN = tf.keras.models.load_model("../Model/Clean_Robust_CNN.h5")

Adversarial Samples

In [8]:
def Adversarial_Samples(Data, y_Data):
    D = np.copy(Data)
    D = np.transpose(D, (0,2,1))
    D = np.expand_dims(D,axis=-1)

    Size = D.shape[0]

    CW = CarliniWagnerL2(
            model_fn = Robust_CNN,
            y = y_Data,
            targeted = False,
            batch_size = Size,
            clip_max = Max,
            clip_min = Min,
            max_iterations = nb_Iterations,
            confidence = Confidence,
            learning_rate = Learning_Rate
        )

    AD = CW.attack(D)
    AD = AD[:,:,:,0]
    Adv_Data = np.copy(np.transpose(AD, (0,2,1)))
    
    return Adv_Data

Adversarial Dataset

In [9]:
def Adversarial_Dataset(Dataset):
    ModulationSchemes = list(Dataset.keys())
    SNRs = list(Dataset[ModulationSchemes[0]].keys())

    Adv_Dataset = {}
    for modType in ModulationSchemes:
        Adv_Dataset[modType] = {}
        for snr in SNRs:
            Adv_Data = np.copy(Dataset[modType][snr])
            y_Data = np.array([ModulationMap[modType]]*Adv_Data.shape[0])
            Adv_Dataset[modType][snr] = Adversarial_Samples(Adv_Data, y_Data)

        print ("Adversarial Data Generated for " + modType)
    return Adv_Dataset

### Generating Adversarial Training Data

In [10]:
# # Training Clean Dataset
# f = open("../Dataset/TrainDataset.pkl", "rb")
# TrainDataset = pickle.load(f)
# f.close()

# # Training Adversarial Dataset
# Adv_TrainDataset = Adversarial_Dataset(TrainDataset)
# f = open("../Dataset/CW_Adv_TrainDataset.pkl", "wb")
# pickle.dump(Adv_TrainDataset, f)
# f.close()

### Generating Adversarial Validation Data

In [11]:
# Validation Clean Dataset
f = open("../Dataset/ValidDataset.pkl", "rb")
ValidDataset = pickle.load(f)
f.close()

# Validation Adversarial Dataset
Adv_ValidDataset = Adversarial_Dataset(ValidDataset)
f = open("../Dataset/CW_Adv_ValidDataset.pkl", "wb")
pickle.dump(Adv_ValidDataset, f)
f.close()

Adversarial Data Generated for 8PSK
Adversarial Data Generated for AM-DSB
Adversarial Data Generated for AM-SSB
Adversarial Data Generated for BPSK
Adversarial Data Generated for CPFSK
Adversarial Data Generated for GFSK
Adversarial Data Generated for PAM4
Adversarial Data Generated for QAM16
Adversarial Data Generated for QAM64
Adversarial Data Generated for QPSK
Adversarial Data Generated for WBFM
